<a href="https://colab.research.google.com/github/rb58853/ML-RSI-Images/blob/main/T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.6 MB/s eta 0:00:00


In [29]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# the following 2 hyperparameters are task-specific
max_source_length = 512
max_target_length = 128

# # Suppose we have the following 2 training examples:
# input_sequence_1 = "Welcome to NYC"
# output_sequence_1 = "Bienvenue à NYC"

# input_sequence_2 = "HuggingFace is a company"
# output_sequence_2 = "HuggingFace est une entreprise"

# # encode the inputs
# task_prefix = "translate English to French: "
# input_sequences = [input_sequence_1, input_sequence_2]

inputs = [
    "a cat playing",
    "a dog playing",
    "a bear playing"
    ]

outputs = [
    "a cat | a cat playing | cat | cat playing",
    "a dog | a dog playing | dog | dog playing",
    "a bear | a bear playing | bear | bear playing"
    ]

encoding = tokenizer(
    inputs,
    padding="longest",
    max_length=max_source_length,
    truncation=True,
    return_tensors="pt",
)

input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

# encode the targets
target_encoding = tokenizer(
    outputs,
    padding="longest",
    max_length=max_target_length,
    truncation=True,
    return_tensors="pt",
)
labels = target_encoding.input_ids

# replace padding token id's of the labels by -100 so it's ignored by the loss
labels[labels == tokenizer.pad_token_id] = -100

# forward pass
loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss
loss.item()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


4.12986421585083

In [32]:
input = tokenizer("a cat playing", return_tensors="pt").input_ids
outputs = model.generate(input)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))